In [131]:
#files to open: train and test data,

from google.colab import files

uploaded = files.upload()


Saving train_0_change_1_to_close.csv to train_0_change_1_to_close.csv


In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.utils import class_weight
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [4]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(epochs, loss, val_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(np.arange(epochs), loss, label='loss')
    plt.plot(np.arange(epochs), val_loss, label='val_loss')
    plt.legend()
    plt.grid(True)
def acc_graph(epochs, train_acc, val_acc):
    x_axis = np.linspace(1, epochs, num=epochs)
    
    plt.figure(figsize=(10, 7))
    plt.plot(x_axis, train_acc, color='g', lw=3, alpha=0.7, label='Train Accuracy')
    plt.plot(x_axis, val_acc, color='orange', lw=3, alpha=0.7, label='Val Accuracy')
    plt.title('Accuracy graph')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

In [0]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [0]:
#from utils import model

In [0]:
from sklearn.preprocessing import MinMaxScaler

def scale_0_1_for_samples(X):
  mm = MinMaxScaler()

  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
  
    line = line.reshape(-1, 1)
    X[i] = mm.fit_transform(line).reshape(1,-1)
  return X

In [0]:
def get_mean_for_features(X):
  m = X.shape[0]
  X_mean = np.zeros((m,1))
  for i in range(m):
    X_mean[i] = np.mean(X[i,:], axis=0)
  return X_mean

def scale_standart_for_samples(X, X_mean):
  ss = StandardScaler()
  ss.fit(X_mean)

  X = ss.transform(X)
  return X

In [0]:
def make_predictions(X, y, model, out=False, X_ini=X_ini):
  preds = model.predict(X)
  y_predict = np.where(preds > 0.5, 1, 0)
  recall = recall_score(y, y_predict)
  print('Recall = ', recall, 'Precision = ', precision_score(y, y_predict))
  cm  = confusion_matrix(y, y_predict)
  print(cm)
  if out is True:
    for i in range(X.shape[0]):
      if abs(y_predict[i] - y[i])!= 0:
        print(y[i], preds[i], X_ini[i], np.ndarray.tolist(X[i]))
    

# Import data sets

In [0]:
#data_train = pd.read_csv('sources/train.csv')
#data_test = pd.read_csv('sources/test_v7_v8.csv')

data_train = pd.read_csv('train.csv')


In [0]:
data_test = pd.read_csv('test_v7_v8.csv')


In [0]:
#data_test = pd.read_csv('sources/test_v3.csv')
data_test2 = pd.read_csv('test_v3.csv')


In [0]:
X = data_train.drop(['Y'], axis='columns').values
y = data_train['Y'].values
X_ini = data_train.drop(['Y'], axis='columns').values

In [0]:
data_add = pd.read_csv('train_0_change_1_to_close.csv')

X_add = data_add.drop(['Y'], axis='columns').values
y_add = data_add['Y'].values


In [181]:
X = np.vstack((X, X_add))
print(X.shape)

y = np.hstack((y, y_add))
print(y.shape)

(71399, 6)


In [183]:
print(np.sum(y))
print(y.shape[0] - np.sum(y))

1823.0
69576.0


In [0]:
X_test = data_test.drop(['Y'], axis='columns').values
test =  data_test.drop(['Y'], axis='columns').values
y_test = data_test['Y'].values

X_test2 = data_test2.drop(['Y'], axis='columns').values
test2 =  data_test2.drop(['Y'], axis='columns').values
y_test2 = data_test2['Y'].values


In [0]:
X = scale_0_1_for_samples(X)
X_test = scale_0_1_for_samples(X_test)
X_test2 = scale_0_1_for_samples(X_test2)

In [0]:
X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               shuffle=True, stratify=y)

class_weights_train = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y), y)


In [187]:
print(class_weights)
print(class_weights[0]/(class_weights[1]+class_weights[0]))
calculation_classes(y, y_dev)

[ 0.51310078 19.5828305 ]
0.02553257048418045
		TRAIN DATA	TEST DATA
Negative	 69576    97.45 % 	 13915    97.44 %
Positive	  1823   2.55 %	 365   2.56 %


In [188]:
print(X_train[0], X_dev[0])

[1.         0.8        0.62068966 0.43103448 0.22413793 0.        ] [0.97382199 0.84293194 0.         0.56544503 0.81151832 1.        ]


In [0]:
#calculation_classes(y, y_dev)
#calculation_classes(y_train, y_dev)

In [190]:
X = np.expand_dims(X, axis=2)
X_train = np.expand_dims(X_train, axis=2)
X_dev = np.expand_dims(X_dev, axis=2)

print(X_test.shape)
X_test = np.expand_dims(X_test, axis=2)
X_test2 = np.expand_dims(X_test2, axis=2)
print(X_test.shape)

(59, 6)
(59, 6, 1)


In [191]:
print(X.shape)

(71399, 6, 1)


In [0]:

#print(y)
X_ini_add = data_add.drop(['Y'], axis='columns').values

X_add = scale_0_1_for_samples(X_add)
X_add= np.expand_dims(X_add, axis=2)

# NN on train/dev sets

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


def model_GRU(input_shape, Ty, hidden_size):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(hidden_size, return_sequences=False)(i)
    
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model


In [115]:
# For ready model
'''GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


GRU_model.load_weights('GRU_sequence_weights.h5')'''

"GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()\nGRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n\n\nGRU_model.load_weights('GRU_sequence_weights.h5')"

In [0]:
Tx=  6
Ty = 1
epochs = 5
batch_size = 32

In [196]:
GRU_model = model_LSTM(16)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
logs = GRU_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train, \
                    initial_epoch=0)

Train on 71399 samples, validate on 14280 samples
Epoch 1/5
 - 11s - loss: 0.0895 - acc: 0.9766 - val_loss: 0.0371 - val_acc: 0.9807
Epoch 2/5
 - 9s - loss: 0.0210 - acc: 0.9916 - val_loss: 0.0162 - val_acc: 0.9933
Epoch 3/5
 - 9s - loss: 0.0099 - acc: 0.9967 - val_loss: 0.0063 - val_acc: 0.9982
Epoch 4/5
 - 9s - loss: 0.0071 - acc: 0.9979 - val_loss: 0.0442 - val_acc: 0.9888
Epoch 5/5
 - 9s - loss: 0.0057 - acc: 0.9982 - val_loss: 0.0061 - val_acc: 0.9977


In [0]:
opt = Adam(lr=0.009, beta_1=0.9, beta_2=0.999 ,decay=0.01)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])



In [199]:
n_epochs = 5
logs = GRU_model.fit(X_train, y_train , epochs=n_epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train)

Train on 57119 samples, validate on 14280 samples
Epoch 1/5
 - 7s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9997
Epoch 2/5
 - 7s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9998
Epoch 3/5
 - 8s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9998
Epoch 4/5
 - 7s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9998
Epoch 5/5
 - 7s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9997


In [203]:
make_predictions(X,y, out=True, model=GRU_model)

Recall =  0.9983543609434997 Precision =  0.989668297988037
[[69557    19]
 [    3  1820]]
0.0 [0.55742085] [-0.08  -0.044 -0.01   0.023  0.033  0.077] [[0.0], [0.2292993630573248], [0.445859872611465], [0.6560509554140127], [0.7197452229299364], [1.0]]
0.0 [0.98009443] [-0.08  -0.044 -0.01   0.023  0.07   0.106] [[0.0], [0.1935483870967742], [0.3763440860215054], [0.553763440860215], [0.8064516129032258], [1.0]]
0.0 [0.55742085] [-0.08  -0.044 -0.01   0.023  0.033  0.077] [[0.0], [0.2292993630573248], [0.445859872611465], [0.6560509554140127], [0.7197452229299364], [1.0]]
0.0 [0.98009443] [-0.08  -0.044 -0.01   0.023  0.07   0.106] [[0.0], [0.1935483870967742], [0.3763440860215054], [0.553763440860215], [0.8064516129032258], [1.0]]
0.0 [0.74899864] [-0.08  -0.044  0.008  0.051  0.07   0.106] [[0.0], [0.1935483870967742], [0.4731182795698925], [0.7043010752688172], [0.8064516129032258], [1.0]]
0.0 [0.8359766] [ 0.055  0.024 -0.01  -0.036 -0.069 -0.087] [[1.0], [0.7816901408450704], [0.

In [0]:
#make_predictions(X_dev,y_dev, out=True, model=GRU_model)

In [201]:
make_predictions(X_test, y_test, out=True, model=GRU_model, X_ini=test)

Recall =  0.9130434782608695 Precision =  1.0
[[13  0]
 [ 4 42]]
1.0 [0.07533211] [3.574 3.574 3.574 3.575 3.578 3.579] [[0.0], [0.0], [0.0], [0.20000000000004547], [0.8000000000000682], [1.0]]
1.0 [2.3931265e-05] [3.574 3.574 3.575 3.578 3.579 3.582] [[0.0], [0.0], [0.125], [0.5], [0.6249999999999432], [1.0]]
1.0 [0.00078446] [3.574 3.575 3.578 3.579 3.582 3.586] [[0.0], [0.08333333333337123], [0.33333333333337123], [0.4166666666666856], [0.6666666666666856], [1.0]]
1.0 [0.00607404] [3.575 3.578 3.579 3.582 3.586 3.592] [[0.0], [0.17647058823530415], [0.23529411764704378], [0.4117647058823479], [0.6470588235294485], [1.0]]


In [202]:
make_predictions(X_test2, y_test2, out=True, model=GRU_model, X_ini=test2)

Recall =  1.0 Precision =  1.0
[[19  0]
 [ 0 53]]


In [0]:
#make_predictions(X_add, y_add, out=True, model=GRU_model, X_ini=X_ini_add)

In [0]:
#plot_loss(epochs, logplot_loss(epos.history.get('loss'), logs.history.get('val_loss'))

In [0]:
#acc_graph(epochs, logs.history.get('acc'), logs.history.get('val_acc'))

In [0]:
GRU_model.save_weights('GRU_sequence_weights.h5')

In [0]:
files.download('GRU_sequence_weights.h5')

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logs = LSTM_model.fit(X_train, y_train , epochs=epochs, batch_size=batch_size, shuffle=True,\
               validation_data=(X_dev, y_dev), verbose=0, class_weight=class_weights_train)

# Final fit NN on all train set

In [0]:
Tx=  6
Ty = 1
epochs = 250
batch_size = 32

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
               verbose=0, class_weight=class_weights)
LSTM_model.save_weights('sources/LSTM_sequence_weights.h5')

In [0]:
print(LSTM_model.summary())
test_preds = LSTM_model.predict(X_test)

In [0]:
for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)

In [0]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
test_preds = LSTM_model.predict(X_test)

for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)